In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from main import read_file, get_sentences
import stanza

stanza.download("en", model_dir="../models/stanza_resources")

2023-04-04 20:03:23 INFO: Downloading default packages for language: en (English) ...
2023-04-04 20:03:29 INFO: File exists: ../models/stanza_resources/en/default.zip
2023-04-04 20:03:38 INFO: Finished downloading models and saved to ../models/stanza_resources.


In [3]:
henry_red = read_file("../data/english_short_stories/Henry_Red_Chief.txt")

preprocess = stanza.Pipeline("en", processors="tokenize, ner, pos, lemma")

doc = preprocess(henry_red)

Could not read file '../data/english_short_stories/Henry_Red_Chief.txt' using the utf-8 encoding
'utf-8' codec can't decode byte 0xd1 in position 123: invalid continuation byte
Could not read file '../data/english_short_stories/Henry_Red_Chief.txt' using the ascii encoding
'ascii' codec can't decode byte 0xd1 in position 123: ordinal not in range(128)
2023-04-04 20:03:38 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-04-04 20:03:40 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| ner       | ontonotes |

2023-04-04 20:03:40 INFO: Using device: cpu
2023-04-04 20:03:40 INFO: Loading: tokenize
2023-04-04 20:03:40 INFO: Loading: pos
2023-04-04 20:03:41 INFO: Loading: lemma
2023-04-04 20:03:41 INFO: Loadin

In [4]:
get_sentences(doc, lambda x: x.upos != "PUNCT")

[
  {
    "id": 1,
    "text": "The",
    "lemma": "the",
    "upos": "DET",
    "xpos": "DT",
    "feats": "Definite=Def|PronType=Art",
    "start_char": 2,
    "end_char": 5,
    "ner": "B-PERSON",
    "multi_ner": [
      "B-PERSON"
    ]
  },
  {
    "id": 2,
    "text": "Ransom",
    "lemma": "Ransom",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "start_char": 6,
    "end_char": 12,
    "ner": "I-PERSON",
    "multi_ner": [
      "I-PERSON"
    ]
  },
  {
    "id": 3,
    "text": "of",
    "lemma": "of",
    "upos": "ADP",
    "xpos": "IN",
    "start_char": 13,
    "end_char": 15,
    "ner": "I-PERSON",
    "multi_ner": [
      "I-PERSON"
    ]
  },
  {
    "id": 4,
    "text": "Red",
    "lemma": "Red",
    "upos": "ADJ",
    "xpos": "NNP",
    "feats": "Degree=Pos",
    "start_char": 16,
    "end_char": 19,
    "ner": "I-PERSON",
    "multi_ner": [
      "I-PERSON"
    ]
  },
  {
    "id": 5,
    "text": "Chief",
    "lemma": "Chief",
    "upos": "PRO